# **Pokémon Diffusion<a id="top"></a>**

> #### ``03-Diffusion-Process.ipynb``

<i><small>**Alumno:** Alejandro Pequeño Lizcano<br>Última actualización: 24/03/2024</small></i></div>

Como bien se ha descrito antes, el modelo de difusión es un modelo generativo que se basa en la difusión de ruido en una imagen para generar muestras. Este modelo se puede dividir en dos etapas: el modelo en sí y el **proceso de difusión**. En el último notebook se explicó la arquitectura del modelo de difusión, por lo que en este notebook se explicará el proceso de difusión que a su vez se divide en dos partes:

- Difusión hacia adelante ***(Forward Diffusion)***
- Difusión inversa ***(Inverse Diffusion)***. Cada una de estas dos partes, simboliza los dos pasos del modelo de difusión: añadir ruido a la imagen de entrada y eliminar el ruido de la imagen resultante. 

Todo ello siguiendo una ***cadena de Markov***, donde el proceso de **forward diffusion** se puede describir como: $p(x_{t} | x_{t-1})$, la distribución de probabilidad de la imagen en el paso de tiempo $t$ dado el paso de tiempo anterior $t-1$. De modo que la imagen en el paso de tiempo $t$ depende solo de la imagen en el paso de tiempo anterior $t-1$. Y el **inverse diffusion** $q_{\theta}(x_{t-1} | x_{t})$ es la distribución de probabilidad inversa, es decir, la distribución de probabilidad de la imagen en el paso de tiempo $t-1$ dado el paso de tiempo $t$ y de igual manera, la imagen en el paso de tiempo $t-1$ depende solo de la imagen en el paso de tiempo $t$.

<p align="center">
    <img src='../figures/notebook_figures/diffusion_overview.png' width=40% height=50% />
</p>

En cada sección, se explicará el funcionamiento de cada una de las partes del modelo de difusión. Primero, empezaremos definiendo los hiperparámetros del modelo de difusión. Estos hiperparámetros son los que se han usado en los papers [Denoising Diffusion Probabilistic Models](https://arxiv.org/abs/2006.11239), [Diffusion Models Beat GANs on Image Synthesis](https://arxiv.org/abs/2105.05233) y [Improved Denoising Diffusion Probabilistic Models](https://arxiv.org/abs/2102.09672) y se han mantenido para este proyecto.

- ``T``: Número de pasos de difusión, es decir, el número de veces que se añade ruido a la imagen de entrada.

- ``beta_start`` y ``beta_end``: Parámetros que determinan la cantidad de ruido que se añade en cada paso. Un valor más alto de *beta* significa que se añade más ruido, y, por tanto, una mayor diversidad en las muestras generadas, pero también implica una mayor dificultad para el modelo de difusión. *beta_start* es el valor de *beta* en el primer paso y *beta_end* es el valor de *beta* en el último paso. Más adelante se explicará este concepto con más detalle.

- ``s``: el factor de escala de la distribución de ruido. Es decir, determina el desplazamiento aplicado al paso de tiempo "t" durante el proceso de añadir ruido a la imagen. Este desplazamiento modifica la fase de la función coseno, influyendo en la progresión de la varianza desde los pasos de tiempo iniciales hasta los finales, afectando así la suavidad de la transición en los niveles de ruido aplicados durante el proceso de difusión.

---

<span style="color: red; font-size: 1.5em;">&#9888;</span> <i><small>**NOTA:** Se ha optado por darle unos valores bajos de beta ya que en este notebook haremos uso del ``cosine`` *scheduler*. Más adelante explicaremos este concepto.
</small></i>

- [0. Imports](#-0.-Imports)

- [1. Dataset](#-1.-Dataset)

    - [1.1. Carga de datos](#-1.1.-Carga-de-datos)
    - [1.2. Creación del dataset](#-1.2.-Creación-del-dataset)

- [2. Modelo](#-2.-Modelo)

- [3. Forward Diffusion](#-2.-Forward-Diffusion)

- [4. Inverse Diffusion](#-3.-Inverse-Diffusion)

# 0. Imports

Una vez introducido el proyecto, se importan las librerías necesarias para el desarrollo de este apartado.

---

In [1]:
# Import necessary libraries
# =====================================================================
import configparser
import tensorflow as tf

# Import src code
from src.model.build_unet import build_unet
from src.utils.utils import PROJECT_DIR, DATA_PATH

2024-03-24 20:02:05.375593: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-24 20:02:05.403657: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-24 20:02:05.880996: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Set config file
# =====================================================================
config = configparser.ConfigParser()
config.read(PROJECT_DIR / "config.ini")

# Hyperparameters
config_hp = config["hyperparameters"]

IMG_SIZE = int(config_hp["img_size"])
NUM_CLASSES = int(config_hp["num_classes"])
BATCH_SIZE = int(config_hp["batch_size"])
EPOCHS = int(config_hp["epochs"])
T = int(config_hp["T"])
BETA_START = float(config_hp["beta_start"])
BETA_END = float(config_hp["beta_end"])
s = float(config_hp["s"])

# 1. Dataset

En primer lugar se cargarán los datos necesarios para la creación del dataset de entrenamiento del modelo de difusión. Por ello, se cargará el csv ya limpio y los paths de imágenes con el objetivo de crear dicho dataset haciendo uso de la función ``create_datatset()`` definida en el notebook anterior, aplicando todo el proceso de preprocesado de datos explicado previamente.


A continuación, haremos uso de la función ``create_dataset()`` definida con anterioridad para crear el dataset de entrenamiento del modelo de difusión y aplicar todo el proceso de preprocesado de datos explicado anteriormente.

In [3]:
dataset_path = DATA_PATH / "processed/pokemon_tf_dataset"

poke_dataset = tf.data.Dataset.load(path=dataset_path, compression="GZIP")

TypeError: TF_DATA_SnapshotMetadataFilePath(): incompatible function arguments. The following argument types are supported:
    1. (arg0: str) -> str

Invoked with: PosixPath('/mnt/c/Users/pqlza/UPM/TFG/DiffusionScratch/data/processed/pokemon_tf_dataset')

In [ ]:
# Check the dataset
# =====================================================================
for img, label in poke_dataset.take(1):
    print(
        f"- The shape of the dataset is: {img.shape} => batch_size: {img.shape[0]}, height: {img.shape[1]}, width: {img.shape[2]}, channels: {img.shape[3]}"
    )
    print(
        f"- The shape of the dataset labels is: {label.shape} => batch_size: {label.shape[0]}, number of labels: {label.shape[1]}"
    )

# Visualize the dataset
visualize.plot_images_batch(poke_dataset, df, 6)

## 1.2. Creación del dataset

In [ ]:
# Hyperparameters
# =====================================================================
T = 1000  # number of diffusion steps
beta_start = 1e-4  # initial value of beta
beta_end = 0.02  # final value of beta
s = 0.008  # scale factor for the variance curve

IMG_SIZE = 64
BATCH_SIZE = 32

# Create the dataset
# =====================================================================
# poke_dataset = create_dataset.dataset_tf(
#     image_paths=image_paths,
#     df=df,
#     img_size=IMG_SIZE,
#     batch_size=BATCH_SIZE,
#     save=True,
#     save_path=f"{data_path}/processed/pokemon_tf_dataset",
# )  # remember that you can also give the data_dict as input instead of the image_paths and df

poke_dataset = tf.data.Dataset.load(
    path=f"{data_path}/processed/pokemon_tf_dataset", compression="GZIP"
)

In [ ]:
# Check the dataset
# =====================================================================
for img, label in poke_dataset.take(1):
    print(
        f"- The shape of the dataset is: {img.shape} => batch_size: {img.shape[0]}, height: {img.shape[1]}, width: {img.shape[2]}, channels: {img.shape[3]}"
    )
    print(
        f"- The shape of the dataset labels is: {label.shape} => batch_size: {label.shape[0]}, number of labels: {label.shape[1]}"
    )

# Visualize the dataset
visualize.plot_images_batch(poke_dataset, df, 6)

# 2. Modelo

In [ ]:
# Load model
# =====================================================================
# model = fd.DiffusionModel(
#     T=T, beta_start=beta_start, beta_end=beta_end, s=s, img_size=IMG_SIZE
# )

In [ ]:
# Create a tensorboard callback for monitoring training progress
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir="./logs", histogram_freq=1
)

# Compile the model
model = build_ddpm_model()
model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

# Train the model
model.fit(x_train, y_train, epochs=10, callbacks=[tensorboard_callback])

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

## Forward Diffusion

El proceso de **forward diffusion** se basa en añadir iterativamente ruido a la imagen
de entrada hasta conseguir ruido puro que siga una distribución gaussiana $\mathcal{N}(0, I)$, donde $I$ es la matriz identidad. Este proceso se realiza en $T$ pasos, donde en cada paso se añade una cantidad de ruido determinada por el parámetro $\beta$ a través de un **variance scheduler** que no es más  que la secuencia de $\beta$ que describe la manera en la que se añade ruido a la imagen de entrada dependiendo del instante de tiempo en el que se encuentre hasta llegar a $T$.

<p align="center">
    <img src='../figures/notebook_figures/forward_diffusion.jpeg' width=50% height=50% />
</p>

Cada instante $t+1$ es generado a partir del instante $t$ añadiendo ruido gaussiano a la imagen. Con esto, se consigue cambiar la distribución de datos de los ejemplos del dataset a una gaussiana. Cada eslabón de la cadena de Markov de nuestro modelo corresponde a una distribución distinta. Es decir, la distribución de ruido es una distribución gaussiana con media 0 y varianza $\beta_{t}$, este valor de $\beta_{t}$ es el **diffusion rate** que se calcula para cada instante de tiempo *t* a partir dependiendo del **schedule** que se use, como ya se menciono con anterioridad. 

Por tanto, la distribución de probabilidad de la imagen en el paso de tiempo $t$ dado el paso de tiempo anterior $t-1$ se puede expresar como:

$$p(x_{t} | x_{t-1}) = \mathcal{N}(x_{t} | \sqrt{1 - \beta_{t}}x_{t-1}, \beta_{t}I)$$

Esta fórmula se puede simplificar a través del truco de reparametrización, el renombramiento de $\beta_{t}$ como $\alpha_{t} = 1 - \beta_{t}$ y $\bar{\alpha}_{t} = \prod_{i=1}^{t} \alpha_{i}$ y por recursividad, podemos reescribir la anterior expresión como:

$$x_{t} = \sqrt{\bar{\alpha}_{t}}x_{0} + \sqrt{1 - \bar{\alpha}_{t}}\epsilon_{t}$$

Con este proceso conseguiremos obtener una imagen en cualquier instante de tiempo $t ,,0 <= t <= T$ de la difusión, para posteriormente poder realizar el proceso inverso.

Para lograr esto, se han definido varias funciones. La función principal, ``forward_diffusion()`` se encarga de realizar la difusión hacia adelante implementando lo comentado con anterioridad. Para ello, primero define el valor $\beta$ a través de ``beta_schedule()``. Esta función se encarga de calcular el valor de $\beta$ en cada paso de difusión. Tenemos dos opciones para calcular el valor de $\beta$:
- ``linear`` que añade ruido de manera lineal.
- ``cosine`` que añade ruido de manera más suave al principio y más abrupta en el medio.
   
En este caso, se ha optado por usar ``cosine`` ya según el paper original, se obtienen mejores resultados. Esto se debe a que la función coseno es más suave que la lineal, por lo que la transición entre los niveles de ruido aplicados en cada paso de difusión es más suave. Esto permite que el modelo de difusión aprenda de manera más eficiente y estable, y, por tanto, genere muestras de mayor calidad, pues a partir de algo más de la mitad de los pasos, niveles de ruido usando ``linear`` son muy altos, es decir ya son imágenes muy ruidosas, y esto hace que el modelo de difusión tenga más dificultad para aprender, pues son redundantes y destruye la información de la imagen original muy rápido.

Para calcular el scheduler del coseno, se usa la siguiente fórmula:

$$\bar{\alpha}_t = \frac{f(t)}{f(0)}, \quad f(t) = \cos \left( \frac{t/T + s}{1 + s} \cdot \frac{\pi}{2} \right)^2,  \beta_t = 1 - \frac{\bar{\alpha}_t}{\bar{\alpha}_{t-1}}.
$$

En la práctica, limitamos a **$\beta$** para que no sea mayor que 0.999 para prevenir singularidades al final del proceso de difusión cerca de $t = T$.

Una vez definido el valor de **$\beta$**, se procede a calcular el valor acumulado de **$\alpha$** en cada paso de difusión y a generar el ruido que se añadirá a la imagen de entrada. Finalmente, se devuelve la imagen de entrada con el ruido añadido. Esto se hace siguiendo la fórmula anteriormente comentada, que nos permite calcular una imagen en un instante de tiempo *t* cualquiera a partir de la imagen original sin necesidad de iterar todo el rato hasta llegar a ese instante de tiempo, lo cual hace que nos ahorremos mucho tiempo de entrenamiento. ($x_{t} = \sqrt{\bar{\alpha}_{t}}x_{0} + \sqrt{1 - \bar{\alpha}_{t}}\epsilon_{t}$)

---

<i><small>**Más información** sobre como se consigue esta fórmula en el artículo [What are Diffusion Models?](https://lilianweng.github.io/posts/2021-07-11-diffusion-models/)

<span style="color: red; font-size: 1.5em;">&#9888;</span> **NOTA:** El proceso matemático para llegar a esta fórmula es muy complejo para explicarlo en este notebook. Sin embargo, en el futuro informe se explicará con más detalle. </small></i>

Para ayudar a visualizar este proceso de difusión directa, usaremos:
- ``plot_noise_levels()`` mustra una gráfica donde se ve como se va añadiendo ruido hasta obtner un ruido total. 
- ``plot_forward_diffusion()`` muestra dicho proceso gráficado pero con un imagen en distintos intasntes de tiempo. 

Esto se hace tanto como para ``linear`` como para ``cosine``. Todo ello, con el objetivo de ilustrar la justificación del uso de ``cosine``.

In [ ]:
# Plot all the noise levels in the same plot
# =====================================================================
visualize.plot_noise_levels(T, beta_start, beta_end, s)

# Plot the forward diffusion function
# =====================================================================
X_train = poke_dataset.map(lambda x, y: x)  # Get the images from the dataset
random_image = next(iter(X_train))[random.randint(0, BATCH_SIZE)]

visualize.plot_forward_diffusion(random_image, "linear", 10, T, beta_start, beta_end, s)
visualize.plot_forward_diffusion(random_image, "cosine", 10, T, beta_start, beta_end, s)

Como hemos dicho anteriormente, uno de los hiperparámetros más importantes del modelo de difusión es el ``scheduler``. Este hiperparámetro determina la forma en la que se añade el ruido en cada paso de difusión. En este caso, se ha optado por usar ``cosine`` ya según el paper original, se obtienen mejores resultados.

In [ ]:
# Choose the beta scheduler (hyperparameter)
# =====================================================================
SCHEDULER = "cosine"

## Inverse Diffusion

El proceso de **inverse diffusion** se encarga de revertir el proceso de forward diffusion, es decir, parte del ruido puro generado en el último paso de la difusión y, paso a paso, reduce este ruido para reconstruir la imagen original o generar nuevas muestras. Este proceso también se realiza en $T$ pasos, pero a la inversa, comenzando desde $t = T$ hasta llegar a $t = 0$.

La idea es aprender una aproximación de la distribución inversa $q(x_{t-1} | x_{t})$ utilizando un modelo parametrizado con una red neuronal. Dado que esta distribución también es gaussiana para valores de $\beta_t$ suficientemente pequeños, podemos parametrizar directamente la media y la varianza de esta distribución y usarla para muestrear el paso previo en la cadena de Markov. Esto es matemáticamente equivalente a predecir el ruido añadido a la imagen en el paso de tiempo $t$ para generar la imagen en el paso de tiempo $t-1$. El porqué del uso de una red neuronal para este proceso, es que, a diferencia del forward process, no podemos usar $q(x_{t-1} | x_{t})$ para revertir el ruido, ya que es incalculable (intratable).

<p align="center">
    <img src='../figures/notebook_figures/inverse_diffusion.webp' width=50% height=50% />
</p>

Para el proceso de reverse diffusion, cada imagen $x_{t-1}$ se genera a partir de la imagen en el instante $t$ aplicando el modelo aprendido para predecir el ruido que fue añadido en el paso de forward diffusion. La fórmula empleada para realizar este proceso se puede definir como:

$$q(x_{t-1} | x_t, x_0) = \mathcal{N}(x_{t-1} | \mu_{\theta}(x_t, t), \Sigma_{\theta}(x_t, t))$$

- Donde $\mu_{\theta}$ y $\Sigma_{\theta}$ son la media y varianza aprendidas por la red neuronal. Tal que:

\begin{align*}
\mu_{\theta}(x_{t}, t) &:= \tilde{\mu}_{t}(x_{t}, x_{0}) \\
\Sigma_{\theta}(x_{t}, t) &:= \tilde{\beta}_{t} I
\end{align*}

El modelo de reverse diffusion se entrena para **minimizar la diferencia** entre la distribución aproximada y la distribución de ruido real, utilizando una variante del ELBO (Evidence Lower Bound) como función de pérdida. Este proceso de aprendizaje se basa en la reconstrucción de la imagen a partir del ruido, optimizando la red neuronal para que sea capaz de revertir efectivamente el proceso de difusión. Sin embargo, a través de las diversas demostraciones matemáticas, se ha demostrado que el proceso de reverse diffusion es equivalente a $-\log p_{\theta}(x_{0})$, es decir, la probabilidad de la imagen original. Por tanto, el proceso de reverse diffusion se puede ver como un proceso de generación de muestras, donde la red neuronal aprende a generar muestras a partir de ruido puro. Sin embargo, $p_{\theta}(x_{0})$ depende de $x_{1, ..., T}$, por lo que es intratable (incalculable). Sin embargo, dada una función incalculable, podemos calcular el máximo de su límite inferiror que si que es computable: $-\log p_{\theta}(x_{0}) \leq -\log p_{\theta}(x_{0}) + D_{KL}(q(x_{1:T}|x_{0})||p_{\theta}(x_{1:T}|x_{0}))$. A partir de esta fórmula y varios pasos matemáticos, se llega a la siguiente fórmula:

$$\mathcal{L}_{\theta} = \mathbb{E}_{q(x_{1:T}|x_{0})}[-\log p_{\theta}(x_{0})] + D_{KL}(q(x_{1:T}|x_{0})||p_{\theta}(x_{1:T}|x_{0}))$$

- A partir de esta fórmula, se puede llegar a la siguiente expresión:

$$\mathcal{L}_{\theta} = \mathbb{E}_{x_{0},\epsilon} \left[ \frac{(1 - \alpha_{t})^{2}}{2\alpha_{t}(1 - \bar{\alpha}_{t})\sigma_{t}^{2}} \left| \epsilon - \epsilon_{\theta}(x_{t}, t) \right|^{2} \right]$$

- Finalmente, podemos simplificar la función de pérdida ignorando el término de ponderación y simplemente comparando el ruido objetivo y el ruido predicho con el error cuadrático medio **(MSE)**, pues empíricamente se obtienen mejores resultados. Por tanto, la función de pérdida final del modelo de reverse diffusion es la siguiente:

$$L^{simple}_t = \mathbb{E}_{t \sim \mathcal[1,T], \mathbf{x}_0, \epsilon_t} \left[ \left\| \epsilon_t - \epsilon_\theta(\mathbf{x}_t, t) \right\|^2 \right]
$$


---
<i><small>**Más infromación** sobre el porqué matemático de la función de pérdida, aunque ya explicado, se puede encontrar en el paper [Denoising Diffusion Probabilistic Models](https://arxiv.org/abs/2006.11239) y una explicación más clara en la página [Diffusion Model Clearly Explained!](https://medium.com/@steinsfu/diffusion-model-clearly-explained-cd331bd41166).

<span style="color: red; font-size: 1.5em;">&#9888;</span> **NOTA:** El proceso matemático para llegar a esta fórmula es muy complejo para explicarlo en este notebook. Sin embargo, en el futuro informe se explicará con más detalle. 
</small></i>

TODO: enlazar la explicación de inverse difusión con los dos algoritmos (sampling y training)

### Sampling

Una vez que hemos establecido todos los pasos para el preprocesamiento de datos y definido la arquitectura del modelo de difusión, avanzamos hacia la etapa de entrenamiento del modelo. En esta fase, hemos desarrollado funciones auxiliares destinadas a visualizar los resultados del modelo a medida que se lleva a cabo el entrenamiento y también para poder visualizar los resultados finales del modelo una vez que se ha completado el entrenamiento.

Iniciamos con la función ``sampling()``, la cual despliega muestras conforme el modelo se entrena y nos proporciona una herramienta fundamental para evaluar y visualizar el rendimiento del modelo a medida que evoluciona a lo largo del tiempo y la difusión inversa. En primer lugar, se establece $\beta$ y los valores correspondientes de $\alpha$ que desempeñan un papel esencial en la difusión inversa. Posteriormente, se inicializa el ruido, dando inicio al proceso a lo largo de los pasos de difusión. Cada iteración implica la normalización del tiempo, la generación de ruido y la predicción del modelo. La imagen final es obtenida después de aplicar la resta del ruido predicho en un instante de tiempo $t$ a la imagen en ese mismo instante, dando como resultado la imagen en el instante $t-1$. Este proceso se repite hasta que se obtiene la imagen original: $x_{0}$.

Dicha función sigue el algoritmo 2 de [Denoising Diffusion Probabilistic Models](https://arxiv.org/abs/2006.11239) y se ha modificado para que sea capaz de generar imágenes condicionadas a una etiqueta. Para ello, se ha añadido el parámetro ``label`` al modelo ``build_ddpm_model()``. Esto permite que el modelo de difusión genere imágenes de un tipo de pokemon concreto.

<div style="text-align:center">
<img src="../figures/notebook_figures/algorithm2_sampling.png" width="40%" height="30%" />
</div>

In [ ]:
# Algorithm 2: Sampling
# =====================================================================
def sampling(
    model: tf.keras.models.Model,
    start_noise: np.ndarray,
    T: int = T,
    scheduler: str = "linear",
    beta_start: float = beta_start,
    beta_end: float = beta_end,
) -> np.ndarray:
    """
    Samples an image from the model.

    :param model: The model to sample from.
    :param start_noise: The noise to start the sampling from.
    :param T: The number of timesteps to sample for.
    :param scheduler: The type of schedule to use. Options are "linear" or "cosine".
    :param beta_start: Starting value of beta.
    :param beta_end: Ending value of beta.
    :return: The sampled image.
    """

    # Get the beta schedule and corresponding alpha values
    beta = beta_scheduler(scheduler, T, beta_start, beta_end)
    alpha = 1.0 - beta
    alpha_cumprod = np.cumprod(alpha)

    # Set the starting noise
    x_t = start_noise  # 1: x_T ~ N(0, I)

    # Reverse the diffusion process
    for t in tqdm(
        reversed(range(1, T)), desc="Sampling", total=T - 1, leave=False
    ):  # 2: for t = T − 1, . . . , 1 do
        # Compute normalized timestep
        normalized_t = np.array([t / T]).reshape(1, -1).astype("float32")
        # Sample z_t
        z = (
            np.random.normal(size=x_t.shape)
            if t > 1
            else np.zeros(x_t.shape).astype("float32")
        )  # 3: z ∼ N(0, I) if t > 1, else z = 0
        # Calculate x_(t-1)
        predicted_noise = model.predict(
            [x_t, normalized_t], verbose=0
        )  # Predict the noise estimate using the model = eps_theta
        x_t = (
            x_t - (1 - alpha[t]) / np.sqrt(1 - alpha_cumprod[t]) * predicted_noise
        ) / np.sqrt(alpha[t]) + np.sqrt(
            beta[t]
        ) * z  # 4: x_(t-1) = (x_t - (1 - alpha_t) / sqrt(1 - alpha_cumprod_t) * eps_theta) / sqrt(alpha_t) + sigma_t * z

    # Return the final sample
    return x_t  # 5: return x_0

Las funciones auxiliares proporcionadas cumplen roles importantes para la evaluación y visualización del modelo de difusión:

- ``generate_em()``: Esta función crea una etiqueta aleatoria para condicionar el modelo de difusión. Genera un vector de ceros de longitud num_classes y asigna el valor 1 en una posición aleatoria dentro del vector.

- ``plot_samples()``: visualiza muestras generadas por el modelo de difusión a partir de las funciones mencionadas anteriormente.

In [ ]:
# Auxiliary functions
# =====================================================================


# Generate a random embedding (label) =====================================================================
def generate_em(num_classes: int = NUM_CLASSES) -> np.ndarray:
    """
    Generates a random embedding (label)
    :param num_classes: The number of classes
    """
    em = np.zeros(num_classes)
    em[np.random.randint(0, num_classes - 1)] = 1
    return em


# Plot samples function =====================================================================
def plot_samples(
    model: tf.keras.models.Model,
    num_samples: int = 2,
    T: int = T,
    scheduler: str = "linear",
    beta_start: float = beta_start,
    beta_end: float = beta_end,
) -> None:
    """
    Plots samples from the model.

    :param model: The model to sample from.
    :param num_samples: The number of samples to plot.
    :param T: The number of timesteps to sample for.
    :param scheduler: The type of schedule to use. Options are "linear" or "cosine".
    :return: The sampled image.
    """

    fig, axs = plt.subplots(
        1, num_samples, figsize=(num_samples * 2, 2)
    )  # Creating a row of subplots

    for i in trange(num_samples, desc="Sample plot", leave=True):
        start_noise = np.random.normal(size=(1, IMG_SIZE, IMG_SIZE, 3)).astype(
            "float32"
        )
        y_label = generate_em().reshape(
            1, 18
        )  # reshape to (1,18) to match the model input
        sample = sampling(
            model, start_noise, y_label, T, scheduler, beta_start, beta_end
        )
        sample = (sample + 1.0) / 2.0  # Scale to [0, 1]
        axs[i].imshow(sample[0])
        axs[i].title.set_text(
            onehot_to_string(y_label[0])
        )  # use the onehot_to_string function described above
        axs[i].axis("off")

    plt.show()

TODO: INTRODUCIR TRAINING PERO PARA EL SIGUIENTE NOTEBOOK

[BACK TO TOP](#top)